In [0]:
#!pip install opencv-python --user
#!pip install keras --user

In [0]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.losses import categorical_crossentropy
from keras import regularizers
from keras.regularizers import l1
import pandas as pd
import numpy as np
import csv
import cv2

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
width, height = 48, 48
image_size = (width, height)
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
MODELPATH = '/content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5'

In [24]:
def process_csv_file(data):
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions

dataset_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/desafioBB/fer2013.csv")

mask_training = (dataset_df['Usage']=="Training")
mask_test = (dataset_df['Usage']=="PublicTest")
mask_valid = (dataset_df['Usage']=="PrivateTest")

train_raw = dataset_df.loc[mask_training].reset_index(drop=True)
test_raw = dataset_df.loc[mask_test].reset_index(drop=True)
valid_raw = dataset_df.loc[mask_valid].reset_index(drop=True)

X_train, y_train = process_csv_file(train_raw)
X_test, y_test = process_csv_file(test_raw)
X_valid, y_valid = process_csv_file(valid_raw)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [25]:
model = Sequential()
model.add(Convolution2D(filters=16, kernel_size=(5, 5), padding='same',
                        name='image_array', input_shape=(width,height,1)))
model.add(BatchNormalization())
model.add(Convolution2D(filters=16, kernel_size=(5, 5),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=32, kernel_size=(5, 5),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=64, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=64, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=128, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=128, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))

model.add(Convolution2D(filters=256, kernel_size=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(filters=num_labels, kernel_size=(3, 3),
                        strides=(2, 2), padding='same'))

model.add(Flatten())
model.add(Activation("softmax"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 48, 48, 16)        416       
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 16)        6416      
_________________________________________________________________
batch_normalization_22 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
activation_11 (Activation)   (None, 24, 24, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 24, 24, 32)       

In [0]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [27]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, early_stopper, checkpointer])

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 23s 793us/step - loss: 1.8183 - acc: 0.2772 - val_loss: 1.6235 - val_acc: 0.3722

Epoch 00001: val_loss improved from inf to 1.62346, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 2/100
28709/28709 [==============================] - 19s 678us/step - loss: 1.6559 - acc: 0.3541 - val_loss: 1.5689 - val_acc: 0.3923

Epoch 00002: val_loss improved from 1.62346 to 1.56893, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 3/100
28709/28709 [==============================] - 19s 658us/step - loss: 1.5836 - acc: 0.3852 - val_loss: 1.4871 - val_acc: 0.4227

Epoch 00003: val_loss improved from 1.56893 to 1.48714, saving model to /content/drive/My Drive/Colab Notebooks/desafioBB/models/model.h5
Epoch 4/100
28709/28709 [==============================] - 19s 678us/step - loss: 1.5392 - acc: 0.4031 - val_loss: 1.4

In [28]:
scores = model.evaluate(np.array(X_valid), np.array(y_valid), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

3589/3589 [==============================] - 1s 150us/step
Loss: 1.1158526754206552
Accuracy: 0.5862357202604908
